In [17]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [18]:
%tensorflow_version 1.x 
import keras
from keras.applications import ResNet50
from keras.models import Sequential
import matplotlib.pyplot as plt
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import cv2
import numpy as np 
from tensorflow.python.framework import graph_util
from keras.layers import *
from keras.optimizers import *
from sklearn.model_selection import train_test_split
import random
from keras.applications.resnet50 import preprocess_input
from keras.models import Sequential
from keras import optimizers
from keras.applications.resnet50 import ResNet50
from keras.models import Model
import keras

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
data1='/content/drive/My Drive/valf/1coal'
data2='/content/drive/My Drive/valf/2coal'
data3='/content/drive/My Drive/valf/3coal'

In [21]:

def data_with_label_others():
    images=[]
    cnt = 0
    for i in tqdm(os.listdir(data1)):
        path=os.path.join(data1,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img1=cv2.resize(img,(64,64))
        
        img2 = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img2=cv2.resize(img2,(224,224))
        images.append([np.array(img1),np.array(img2),np.array([1,0,0])])
    cnt = 0
    for i in tqdm(os.listdir(data2)):
        path=os.path.join(data2,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img1=cv2.resize(img,(64,64))
        
        img2 = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img2=cv2.resize(img2,(224,224))
        images.append([np.array(img1),np.array(img2),np.array([0,1,0])])
    cnt = 0
    for i in tqdm(os.listdir(data3)):
        path=os.path.join(data3,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img1=cv2.resize(img,(64,64))
        
        img2 = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        img2=cv2.resize(img2,(224,224))
        images.append([np.array(img1),np.array(img2),np.array([0,0,1])])
        
    return images

data = data_with_label_others()

random.shuffle(data)

xo = np.array([i[0] for i in data]).reshape(-1,64,64,1)
xo = xo/255
xr = np.array([i[1] for i in data]).reshape(-1,224,224,3)
xr = preprocess_input(xr)
yo = np.array([i[2] for i in data])



100%|██████████| 216/216 [00:07<00:00, 29.42it/s]


In [22]:
# Loads the weights
def create_model_cnn():
  model=Sequential()
  model.add(InputLayer(input_shape=[64,64,1]))

  model.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=5,padding='same'))
          
  model.add(Conv2D(filters=32,kernel_size=5,strides=2,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=5,padding='same'))
          
  model.add(Conv2D(filters=64,kernel_size=3,strides=2,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=8,padding='same'))

  model.add(Conv2D(filters=128,kernel_size=3,strides=1,padding='same',activation='relu'))
  model.add(MaxPool2D(pool_size=5,padding='same'))
          

  model.add(Flatten())


  model.add(Dense(2048,activation='relu'))
  model.add(Dropout(0.50))
  model.add(Dense(1024,activation='relu'))
  model.add(Dropout(0.50))
  model.add(Dense(512,activation='relu'))
  model.add(Dropout(0.50))
  model.add(Dense(3,activation='softmax'))
  optimizer=Adam(lr=1e-3)
          
  history1=model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
  return model

checkpoint_path = "/content/drive/My Drive/m1.ckpt"
model_cnn=create_model_cnn()

model_cnn.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model_cnn.evaluate(xo, yo, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

Restored model, accuracy: 83.15%


In [28]:
def create_model_resnet():
    
    resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
    model = Sequential()
    model.add(resnet)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(.25))
    model.add(BatchNormalization())
    model.add(Dense(3, activation='softmax'))
    for layer in resnet.layers:
        if isinstance(layer, BatchNormalization):
            layer.trainable = True
        else:
            layer.trainable = False
    model.compile(loss='categorical_crossentropy',optimizer=optimizers.RMSprop(lr=1e-5),metrics=['accuracy'])
    return model
        
  
#ye88 ensem, 75 res
checkpoint_path = "/content/drive/My Drive/kk.ckpt"
model_resnet=create_model_resnet()

model_resnet.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model_resnet.evaluate(xr,  yo, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Restored model, accuracy: 91.95%


In [29]:
#taking two images to check their similarity
def create_model_siamese():

    left_input = Input((64,64,1))
    right_input = Input((64,64,1))

# Base network architecture

    convnet=Sequential()
    convnet.add(InputLayer(input_shape=[64,64,1]))
    convnet.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
    convnet.add(MaxPool2D(pool_size=5,padding='same'))   
    convnet.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
    convnet.add(MaxPool2D(pool_size=5,padding='same'))  
    convnet.add(Conv2D(filters=64,kernel_size=3,strides=1,padding='same',activation='relu'))
    convnet.add(MaxPool2D(pool_size=8,padding='same'))
    convnet.add(Conv2D(filters=128,kernel_size=3,strides=1,padding='same',activation='relu'))
    convnet.add(MaxPool2D(pool_size=5,padding='same'))   
    convnet.add(Conv2D(filters=256,kernel_size=3,strides=1,padding='same',activation='relu'))
    convnet.add(Flatten())

    convnet.add(Dense(1024,activation='relu'))
    convnet.add(Dropout(0.20))
    convnet.add(Dense(512,activation='relu'))
    convnet.add(Dropout(0.25))
    convnet.add(Dense(64,activation='sigmoid'))

#generating the encodings for two input images
    encoded_l = convnet(left_input)
    encoded_r = convnet(right_input)

# Getting the L1 Distance between the 2 encodings
    L1_layer = Lambda(lambda tensor:K.abs(tensor[0] - tensor[1]))

# Add the distance function to the network
    L1_distance = L1_layer([encoded_l, encoded_r])

    prediction = Dense(1,activation='sigmoid')(L1_distance)
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)

    optimizer = Adam(0.0001, decay=2.5e-4)
    history3=siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=['accuracy'])
    return siamese_net





test_left = []
test_right = []
test_targets = []
img1 = xo[95]

a1= cv2.imread("/content/drive/My Drive/fintest/1coal/pic1crop.jpg", cv2.IMREAD_GRAYSCALE)
a1= cv2.resize(a1,(64,64))
a1=np.array(a1).reshape(-1,64,64,1)
a1= a1/255
    
a2= cv2.imread("/content/drive/My Drive/fintest/2coal/pic1crop.jpg", cv2.IMREAD_GRAYSCALE)
a2= cv2.resize(a2,(64,64))
a2=np.array(a2).reshape(-1,64,64,1)
a2= a2/255
    
a3= cv2.imread("/content/drive/My Drive/fintest/3coal/pic1crop.jpg", cv2.IMREAD_GRAYSCALE)
a3= cv2.resize(a3,(64,64))
a3=np.array(a3).reshape(-1,64,64,1)
a3= a3/255

for i in range(yo.shape[0]):
        
    img=np.array(xo[i]).reshape(-1,64,64,1)
    test_left.append(a1)
    test_right.append(img)
    if np.argmax(yo[i]) == 0:
        test_targets.append(1.)
    else:
        test_targets.append(0.)
        
        
    test_left.append(a2)
    test_right.append(img)
    if np.argmax(yo[i]) == 1:
        test_targets.append(1.)
    else:
        test_targets.append(0.)
        
        
    test_left.append(a3)
    test_right.append(img)
    if np.argmax(yo[i]) == 2:
        test_targets.append(1.)
    else:
        test_targets.append(0.)
        


test_left = np.squeeze(np.array(test_left))
test_right = np.squeeze(np.array(test_right))
test_targets = np.squeeze(np.array(test_targets))

test_left = np.array([i for i in test_left]).reshape(-1,64,64,1)
test_right = np.array([i for i in test_right]).reshape(-1,64,64,1)
print(test_left.shape)

model_siamese = create_model_siamese()
# Loads the weights
checkpoint_path='/content/drive/My Drive/m2.ckpt'
model_siamese.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model_siamese.evaluate([test_left,test_right], test_targets, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

(1602, 64, 64, 1)
Restored model, accuracy: 89.83%


In [0]:
def ensemble():
    p=0
    a1= cv2.imread("/content/drive/My Drive/fintest/1coal/pic1crop.jpg", cv2.IMREAD_GRAYSCALE)
    a1= cv2.resize(a1,(64,64))
    a1=np.array(a1).reshape(-1,64,64,1)
    a1= a1/255
    
    a2= cv2.imread("/content/drive/My Drive/fintest/2coal/pic1crop.jpg", cv2.IMREAD_GRAYSCALE)
    a2= cv2.resize(a2,(64,64))
    a2=np.array(a2).reshape(-1,64,64,1)
    a2= a2/255
    
    a3= cv2.imread("/content/drive/My Drive/fintest/3coal/pic1crop.jpg", cv2.IMREAD_GRAYSCALE)
    a3= cv2.resize(a3,(64,64))
    a3=np.array(a3).reshape(-1,64,64,1)
    a3= a3/255
  
    for i in range(yo.shape[0]):
        o=[]
        img=np.array(xo[i]).reshape(-1,64,64,1)
        a=model_cnn.predict(img)
        o.append(np.argmax(a))
        img=np.array(xr[i]).reshape(-1,224,224,3)
        b=model_resnet.predict(img)
        o.append(np.argmax(b))
        
        img=np.array(xo[i]).reshape(-1,64,64,1)
        b1= model_siamese.predict([a1,img])
        b2= model_siamese.predict([a2,img])
        b3= model_siamese.predict([a3,img])
        
        if(b1>=b2 and b1>=b3):
            o.append(0)
        elif (b2>=b1 and b2>=b3):
            o.append(1)
        else:
            o.append(2)
        fin=np.array([0.,0.,0.])
       

        fin[0]=0.25*a[0][0]+0.45*b[0][0]+0.3*b1
        fin[1]=0.25*a[0][1]+0.45*b[0][1]+0.3*b2
        fin[2]=0.25*a[0][2]+0.45*b[0][2]+0.3*b3
        print(fin[0],fin[1],fin[2])

      
        print(np.argmax(fin))
        if(np.argmax(fin)==np.argmax(yo[i])):
            p+=1
            
    print("Ensemble accuracy ",p/yo.shape[0]*100)
        
        
        

In [74]:
ensemble()

0.0008152642985805869 0.012075098231434822 0.8732720613479614
2
0.018036091700196266 0.057957202196121216 0.8814928531646729
2
0.5550740361213684 0.16303783655166626 0.16046260297298431
0
0.018688058480620384 0.867102324962616 0.0016911944840103388
1
0.8521044254302979 0.014967267401516438 0.022072535008192062
0
0.027658378705382347 0.8471734523773193 0.010382848791778088
1
0.8032070398330688 0.05546334758400917 0.011203600093722343
0
0.0047598425298929214 0.046531904488801956 0.858049750328064
2
0.000286946480628103 0.0001843964564613998 0.93972247838974
2
0.0024903796147555113 0.0007166522555053234 0.8880048990249634
2
0.0008183630416169763 0.0059306006878614426 0.8847548961639404
2
0.004922264721244574 0.00840848684310913 0.8893358111381531
2
0.015899358317255974 0.008020073175430298 0.8761764764785767
2
0.0030588095542043447 0.2971816658973694 0.5789758563041687
2
0.0008003198308870196 0.8598335981369019 0.0070569561794400215
1
0.8754917979240417 0.0007343624019995332 0.00015880599